In [ ]:
# loging to Sagemaker ECR with Deep Learning Containers
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-east-1.amazonaws.com
# loging to your private ECR
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 941656036254.dkr.ecr.us-east-1.amazonaws.com

In [ ]:
!./build_and_push.sh tf-from-scratch latest 3_sources/Dockerfile.inference

In [ ]:
! pwd

In [ ]:
import sagemaker, boto3
from sagemaker import get_execution_role

session = sagemaker.Session()
region = session.boto_region_name
role = get_execution_role()
account = boto3.client('sts').get_caller_identity().get('Account')
bucket = session.default_bucket()

model_name="vgg16-model"
endpoint_name= model_name+"-mms-endpoint"
tag = "v1"
image_uri = f"{account}.dkr.ecr.{region}.amazonaws.com/{model_name}:{tag}"

In [ ]:
from sagemaker import Model

mms_model = Model(
    image_uri="941656036254.dkr.ecr.us-east-1.amazonaws.com/tf-from-scratch:latest",
    model_data=None,
    role=role,
    name=model_name,
    sagemaker_session=session
)


In [ ]:
mms_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.xlarge", 
    endpoint_name="test"
)

In [ ]:
! wget https://farm1.static.flickr.com/56/152004091_5bfbc69bb3.jpg

In [ ]:
%matplotlib inline
import cv2
import numpy as np
from matplotlib import pyplot as plt


img = cv2.imread('152004091_5bfbc69bb3.jpg')
resized_img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
resized_filename = "resized_image.jpg"

cv2.imwrite(resized_filename, resized_img)

plt.imshow(cv2.imread(resized_filename))
plt.show()

In [ ]:
import boto3

client = boto3.client('sagemaker-runtime')
accept_type = "application/json"
content_type = 'image/jpeg'
headers = {'content-type': content_type}
payload = open(resized_filename, 'rb')

response = client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=payload,
    ContentType=content_type,
    Accept = accept_type
)


most_likely_label = response['Body'].read()

print(most_likely_label)